### k3d-util.pyとframe_writer.pyを同じディレクトリに格納しておくこと。

In [ ]:
import os, glob, time
import numpy as np
import cv2
from PIL import Image
Image.MAX_IMAGE_PIXELS = 2560*2160*10    # 画像サイズが大きいときはここの数値を大きくする

In [ ]:
import k3d

In [ ]:
import k3d_util_1 as util

In [ ]:
# データの格納場所
data_path = './1ch_spheroid_volume_data.npz'

# 対物レンズ倍率
util.MAGNIFICATION = 60

# 撮影時のビニング
util.BINNING = 1

# Z間隔
util.Z_SLICE_LENGTH = 0.3

# 画像の画素数を減らすための画像縮小率
util.REDUCE_RATIO = 24

# Zスタックの枚数を減らすためのZステップ間隔
util.Z_STEP = 8

In [ ]:
# CV8000でZを3桁撮るとファイル名が桁あふれしてしまうため、正規表現でZ番号を取得する
import re
def z_number(file_name):
    m = re.search(r'Z(\d+)[^Z]*', file_name)

# 3D画像データの読み込み関数の定義
def load_data(file_list):
    file_list.sort(key=z_number)
    img = Image.open(file_list[0])
    buf = np.zeros((len(file_list[::util.Z_STEP]), (img.size[1]//util.REDUCE_RATIO)&~1, (img.size[0]//util.REDUCE_RATIO)&~1), dtype=np.float32)
    for z, i in enumerate(file_list[::util.Z_STEP]):
        img = Image.open(i)
        img = np.asarray(img, dtype=np.float32)
        # y軸の正方向が画面奥になるため(通常2D画像はy軸の正方向は下)、::-1で行を反転する
        img = cv2.resize(img[::-1, :], (buf.shape[2], buf.shape[1]), interpolation=cv2.INTER_AREA)
        buf[z, :, :] = img
    return buf

In [ ]:
#チャネルごとに3D画像データを読み込む
#data1 = load_data(glob.glob(os.path.join(data_path, 'W0040*C1.tif')))
#data2 = load_data(glob.glob(os.path.join(data_path, 'W0040*C2.tif')))
#np.savez_compressed(data_path, data1=data1, data2=data2)
saved_data = np.load(data_path)
data1 = saved_data['data1']

In [ ]:
# 3D画像データの寸法を計算する
util.init_bounds(data1)

In [ ]:
# volume_listに画像データ(複数チャネル)
obj = k3d.volume(
    volume=data1.astype(np.float16), 
    color_map=np.array(k3d.basic_color_maps.BlackBodyRadiation, dtype=np.float32),
    bounds=(0, util.cx, 0, util.cy, 0, util.cz)
)

In [ ]:
# 動画生成時の不具合対策
text_obj = k3d.text2d(text='-', label_box=False, color=0xcccccc, size=.1, position=(0, 1), reference_point='lb', is_html=True)

In [ ]:
# 3Dビューアの表示エリアを生成
plot = k3d.plot(background_color=0xffffff, grid_visible=True, lighting=1.3, height=540)
plot += obj
plot += text_obj

In [ ]:
# コントロールの準備
util.init_controls(plot, obj, text_obj, phi=80, distance=(330, 1, 600))
fov = 50

In [ ]:
# 3Dビューア及びコントロールの表示
plot.display()
util.display_controls() 

# コントロールの初期値を変更
util.slider_ch.value = [170, 1000]

In [ ]:
# FOVやカメラ位置を3D表示に反映する (いったん3D表示してからでないと働かないことがあるので、ここで実行する)
util.refresh()
plot.camera_fov = fov

### 動画作成はこちら👇
* 動画作成にはffmpeg・ffmpeg-python・frame_writer.pyが必要。

In [ ]:
util.display_movie_controls()

In [ ]:
util.input_movie_filename.value = '1ch_spheroid_volume_30fps.mp4'

In [ ]:
util.state_store

### 以下はシーケンスデータのサンプル

In [ ]:
util.state_store = [{'duration': 0,
  'camera_pos': (0,0,0,0,0,0,0,0,0),
  'ox': 0.0,
  'oy': 0.0,
  'oz': 0.0,
  'axis': 'z',
  'ha': 0.0,
  'va': 1.0,
  'd': 600.0,
  'check_ch': True,
  'slider_ch': (170, 1000),
  'plane_x': (0, 82),
  'plane_y': (0, 82),
  'plane_z': (0, 82)},
 {'duration': 6,
  'camera_pos': (0,0,0,0,0,0,0,0,0),
  'ox': 0.0,
  'oy': 0.0,
  'oz': 0.0,
  'axis': 'z',
  'ha': 360.0,
  'va': 90.0,
  'd': 250.0,
  'check_ch': True,
  'slider_ch': (170, 1000),
  'plane_x': (0, 82),
  'plane_y': (0, 82),
  'plane_z': (0, 82)}]

### スクリーンショットはalpha_blending==Trueのときは正常に得られないので注意。

In [ ]:
plot.fetch_screenshot()

In [ ]:
# ブラウザからサーバのPythonへ転送するのに時間がかかるため、0バイトの場合がある。
len(plot.screenshot)

In [ ]:
import base64, io
png_stream = base64.b64decode(plot.screenshot)
img = Image.open(io.BytesIO(png_stream))
img